## Computantional Theory Problems | Nathan Buyrchiyev

In [20]:
import numpy as np

## Problem 1: Binary Words and Operations


### Section 1.1 Parity

The parity function is a simple bitwise operation that shows up in some cryptographic hash functions like SHA-1.
It takes three 32-bit values (x, y, and z) and compares their bits one by one.
For each bit position, it returns 1 if an odd number of the bits are 1 — and 0 otherwise.

So basically, it’s checking whether the number of 1s across the three inputs is odd.
That’s why it’s called a “parity” function — it’s about whether the bits have even or odd parity.

In math form, it’s written as:

Parity(x,y,z)=x⊕y⊕z

The ⊕ symbol just means XOR (exclusive OR). XOR itself already behaves like a parity check:

If you XOR two bits and both are the same (0⊕0 or 1⊕1), you get 0.

If they’re different (0⊕1 or 1⊕0), you get 1.

So when you extend that to three inputs, XOR effectively gives you 1 whenever an odd number of bits are set.

In hash functions, this kind of operation helps mix bits together — it’s good for diffusion, meaning that flipping even one input bit can completely change the output. It also adds a bit of non-linearity, which makes the function harder to reverse-engineer.

I’m using NumPy’s 32-bit unsigned integer type (np.uint32) here so that it behaves more like a low-level operation, similar to how it would in C or in an actual hash algorithm implementation.

This implementation mirrors how the **Parity** function is defined in the Secure Hash Standard (FIPS PUB 180-4).  

In [21]:
def parity(x, y, z):
    """
    Calculates the bitwise XOR of three 32-bit unsigned integers.

    This function takes three numbers, converts them to 32-bit unsigned
    integers, and then computes their bitwise XOR. This operation is
    equivalent to calculating the parity for each bit position across
    the three numbers.

    Args:
        x: The first integer.
        y: The second integer.
        z: The third integer.

    Returns:
        A 32-bit unsigned integer representing the bitwise XOR of x, y, and z.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return x ^ y ^ z

In [22]:

print("--- Basic Tests ---")
# 1 (01) ^ 2 (10) = 3 (11). 3 (11) ^ 3 (11) = 0 (00). Expected: 0x0
test_1 = parity(1, 2, 3) 
print(f"Parity(1, 2, 3) = {hex(test_1)}")

# 0 ^ 0 = 0. 0 ^ 0 = 0. Expected: 0x0
test_2 = parity(0, 0, 0)
print(f"Parity(0, 0, 0) = {hex(test_2)}")

print("\n--- Same Value Tests ---")
# 5 ^ 5 = 0. 0 ^ 5 = 5. Expected: 0x5
test_3 = parity(5, 5, 5)
print(f"Parity(5, 5, 5) = {hex(test_3)}")

# The two 10s cancel out. 10 ^ 10 = 0. 0 ^ 20 = 20. Expected: 0x14
test_4 = parity(10, 20, 10)
print(f"Parity(10, 20, 10) = {hex(test_4)}")

print("\n--- Hash-like Value Tests ---")
# Using large integers that resemble hash values
val_1 = 0xdeadbeef
val_2 = 0xbadf00d
val_3 = 0x12345678
test_5 = parity(val_1, val_2, val_3)
print(f"Parity({hex(val_1)}, {hex(val_2)}, {hex(val_3)}) = {hex(test_5)}")

# Using Python's hash() function, mask to 32 bits to avoid OverflowError
h1 = hash("hello") & 0xFFFFFFFF
h2 = hash("world") & 0xFFFFFFFF
h3 = hash("parity") & 0xFFFFFFFF
test_6 = parity(h1, h2, h3)
print(f"Parity(hash('hello'), hash('world'), hash('parity')) = {hex(test_6)}")

print("\n--- Binary Value Tests ---")
# Using binary literals
# 0b101 (5) ^ 0b010 (2) = 0b111 (7). 0b111 (7) ^ 0b111 (7) = 0. Expected: 0x0
test_7 = parity(0b101, 0b010, 0b111)
print(f"Parity(0b101, 0b010, 0b111) = {hex(test_7)}")

# 1 ^ 1 = 0. 0 ^ 0 = 0. Expected: 0x0
test_8 = parity(1, 1, 0)
print(f"Parity(1, 1, 0) = {hex(test_8)}")

--- Basic Tests ---
Parity(1, 2, 3) = 0x0
Parity(0, 0, 0) = 0x0

--- Same Value Tests ---
Parity(5, 5, 5) = 0x5
Parity(10, 20, 10) = 0x14

--- Hash-like Value Tests ---
Parity(0xdeadbeef, 0xbadf00d, 0x12345678) = 0xc734189a
Parity(hash('hello'), hash('world'), hash('parity')) = 0xb7c0645a

--- Binary Value Tests ---
Parity(0b101, 0b010, 0b111) = 0x0
Parity(1, 1, 0) = 0x0


In [23]:
def ch(x, y, z):
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return (x & y) ^ (~x & z)

In [24]:
print("--- Basic Tests ---")
# For each bit: if x = 1, take y’s bit; if x = 0, take z’s bit.
# x = 1 (01), y = 2 (10), z = 3 (11)
# Bitwise: 01 → choose y’s bit (10) for lower bit, z’s bit (11) for upper bit → Expected: 0x2
test_1 = ch(1, 2, 3)
print(f"Ch(1, 2, 3) = {hex(test_1)}")

# If x = 0, we always choose z. Expected: z = 0x5
test_2 = ch(0, 10, 5)
print(f"Ch(0, 10, 5) = {hex(test_2)}")

# If x = all 1s, we always choose y. Expected: y = 0x9
test_3 = ch(0xFFFFFFFF, 9, 12)
print(f"Ch(0xFFFFFFFF, 9, 12) = {hex(test_3)}")

print("\n--- Same Value Tests ---")
# If y and z are the same, Ch always returns that value, no matter what x is.
# Expected: 0xA
test_4 = ch(7, 10, 10)
print(f"Ch(7, 10, 10) = {hex(test_4)}")

# If x alternates bits (0b1010) and y/z are opposites,
# result should pick alternating bits: 0b1010 from y=0b1111, z=0b0000 → Expected: 0b1010 (0xA)
test_5 = ch(0b1010, 0b1111, 0b0000)
print(f"Ch(0b1010, 0b1111, 0b0000) = {hex(test_5)}")

print("\n--- Hash-like Value Tests ---")
# Using large integers to simulate hash-like inputs
val_1 = 0xdeadbeef
val_2 = 0xbadf00d
val_3 = 0x12345678
test_6 = ch(val_1, val_2, val_3)
print(f"Ch({hex(val_1)}, {hex(val_2)}, {hex(val_3)}) = {hex(test_6)}")

# Using Python hash() masked to 32 bits
h1 = hash("choose") & 0xFFFFFFFF
h2 = hash("function") & 0xFFFFFFFF
h3 = hash("test") & 0xFFFFFFFF
test_7 = ch(h1, h2, h3)
print(f"Ch(hash('choose'), hash('function'), hash('test')) = {hex(test_7)}")

print("\n--- Binary Value Tests ---")
# x = 0b101, y = 0b111, z = 0b000 → picks bits from y when x=1 → Expected: 0b101 (0x5)
test_8 = ch(0b101, 0b111, 0b000)
print(f"Ch(0b101, 0b111, 0b000) = {hex(test_8)}")

# x = 0b010, y = 0b111, z = 0b000 → only middle bit from y → Expected: 0b010 (0x2)
test_9 = ch(0b010, 0b111, 0b000)
print(f"Ch(0b010, 0b111, 0b000) = {hex(test_9)}")


--- Basic Tests ---
Ch(1, 2, 3) = 0x2
Ch(0, 10, 5) = 0x5
Ch(0xFFFFFFFF, 9, 12) = 0x9

--- Same Value Tests ---
Ch(7, 10, 10) = 0xa
Ch(0b1010, 0b1111, 0b0000) = 0xa

--- Hash-like Value Tests ---
Ch(0xdeadbeef, 0xbadf00d, 0x12345678) = 0xabdf01d
Ch(hash('choose'), hash('function'), hash('test')) = 0x44667c73

--- Binary Value Tests ---
Ch(0b101, 0b111, 0b000) = 0x5
Ch(0b010, 0b111, 0b000) = 0x2


## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding


## Problem 4: Hashes


## Problem 5: Passwords


## End